<a href="https://colab.research.google.com/github/bbchen33/Deep-learning/blob/master/PySpark_DeepLearning_TitanicData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install openjdk-8-jdk
!sudo update-alternatives --config java

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 8 newly installed, 0 to remove and 32 not upgraded.
Need to get 5,091 kB of archives.
After this operation, 13.4 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main am

In [2]:
!java -version

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)


In [6]:
!pip install pyspark
!pip install elephas
!pip install -U PyYAML

     |████████████████████████████████| 215.7MB 63kB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=104a182a60e61e025300df2ef3d9cfda6b732986bcadf09ce3b1c68498dfbeb4
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 266kB 2.8MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44104 sha256=7519f8c0fbcdd26723ad5e4ba77328ef1bae3556937035fe0a3f358290b86192
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import files
file = files.upload()

Saving titanic.zip to titanic.zip


In [4]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: train.csv               
  inflating: test.csv                
  inflating: gender_submission.csv   


In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext


In [0]:
conf = SparkConf().setAppName('Spark_DeepLearning_Titanic').setMaster('local')
sc = SparkContext(conf=conf)
sql_context = SQLContext(sc)

The test data does not have label. I'll split the train data into train and validation data to test the model accuracy since we can't use the test data to do so. 

In [0]:
train_df = sql_context.read.csv('train.csv',
                    header=True,
                    inferSchema=True)
test_df = sql_context.read.csv('test.csv',
                    header=True,
                    inferSchema=True)

In [10]:
train_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
train_df.select('survived').distinct().show()

+--------+
|survived|
+--------+
|       1|
|       0|
+--------+



In [12]:
test_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



Describe() provides insight into the type of data under each column. If the mean is null, it's likely to be a categorical column. 

In [13]:
train_df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [14]:
test_df.describe().show()

+-------+------------------+------------------+--------------------+------+------------------+------------------+------------------+------------------+------------------+-----+--------+
|summary|       PassengerId|            Pclass|                Name|   Sex|               Age|             SibSp|             Parch|            Ticket|              Fare|Cabin|Embarked|
+-------+------------------+------------------+--------------------+------+------------------+------------------+------------------+------------------+------------------+-----+--------+
|  count|               418|               418|                 418|   418|               332|               418|               418|               418|               417|   91|     418|
|   mean|            1100.5|2.2655502392344498|                null|  null|30.272590361445783|0.4473684210526316|0.3923444976076555|223850.98986486485|  35.6271884892086| null|    null|
| stddev|120.81045760473994|0.8418375519640503|                null|  

Just to be sure that I put the columns in the right category (categorical or numerical), I look at the possible entries under these columns. 

In [15]:
test_df.select('Ticket').distinct().show()

+---------+
|   Ticket|
+---------+
|   363611|
|     2696|
|    26360|
|   237734|
|   345763|
|    28034|
|    19877|
|C.A. 2673|
|C.A. 2315|
|   220844|
|   366713|
|   350410|
|     1601|
|   237670|
|   364498|
|    21228|
|    28220|
|   347090|
| PC 17757|
|  3101266|
+---------+
only showing top 20 rows



In [16]:
test_df.select('Cabin').distinct().show()

+-----------+
|      Cabin|
+-----------+
|         C6|
|         A9|
|        D28|
|        C78|
|    B58 B60|
|        D21|
|         F2|
|        E31|
|        C31|
|          F|
|    C22 C26|
|       C130|
|B51 B53 B55|
|        A21|
|        A18|
|        E50|
|       null|
|        E45|
|        D34|
|        D37|
+-----------+
only showing top 20 rows



From describe(), one can see that some of the age values are missing. The rows with null columns should be removed. Less than 1/4 of the data had the cabin information. It is not worth using as a feature. 

In [0]:
train_df = train_df.filter(train_df.Age.isNotNull())
train_df = train_df.filter(train_df.Embarked.isNotNull())

In [18]:
train_df.count()

712

In [0]:
test_df = test_df.filter(test_df.Age.isNotNull())
test_df = test_df.filter(test_df.Fare.isNotNull())

In [20]:
test_df.count()

331

In [0]:
categorical_columns = ['Pclass', 'Sex', 'Ticket', 'Embarked']
numeric_columns = ['Age', 'SibSp', 'Parch', 'Fare']

StringIndexer turned categories into numbers. OneHotEncoderEstimator turned the numbers for each category into a list of values like (1, 0) or (0, 1) if there are two categories in this original categorical column.

Then VectorAssembler groups all the features into a vector, a necessary step for machine learning in PySpark.

Finally, Normalizer is used to normalize all the features since they are kind of widely distributed.

In [0]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, \
                               Normalizer, VectorAssembler
transform_columns_steps = []
for item in categorical_columns:
  Index_strings = StringIndexer(inputCol = item, outputCol = f'indexed_{item}')
  Encode_index = OneHotEncoderEstimator(inputCols = [Index_strings.getOutputCol()], outputCols = [f'encoded_{item}'])
  transform_columns_steps += [Index_strings, Encode_index]


all_features = [f'encoded_{item}' for item in categorical_columns] + numeric_columns

vectorize_all_features = VectorAssembler(inputCols = all_features, outputCol = 'unscaled_features')

scale_all_features = Normalizer(inputCol = vectorize_all_features.getOutputCol(), outputCol = 'features')
transform_columns_steps += [vectorize_all_features, scale_all_features]

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = transform_columns_steps)
pipeline_train_model = pipeline.fit(train_df)
transformed_train_df1 = pipeline_train_model.transform(train_df)
transformed_test_df1 = pipeline_train_model.transform(test_df)


In [24]:
transformed_train_df1.take(1)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S', indexed_Pclass=0.0, encoded_Pclass=SparseVector(2, {0: 1.0}), indexed_Sex=0.0, encoded_Sex=SparseVector(1, {0: 1.0}), indexed_Ticket=202.0, encoded_Ticket=SparseVector(540, {202: 1.0}), indexed_Embarked=0.0, encoded_Embarked=SparseVector(2, {0: 1.0}), unscaled_features=SparseVector(549, {0: 1.0, 2: 1.0, 205: 1.0, 543: 1.0, 545: 22.0, 546: 1.0, 548: 7.25}), features=SparseVector(549, {0: 0.043, 2: 0.043, 205: 0.043, 543: 0.043, 545: 0.9454, 546: 0.043, 548: 0.3115}))]

In [25]:
transformed_train_df1.count()

712

In [0]:
index_label = StringIndexer(inputCol = 'Survived', outputCol = 'label')
labeled_train_df1 = index_label.fit(transformed_train_df1).transform(transformed_train_df1)

In [27]:
labeled_train_df1.head()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S', indexed_Pclass=0.0, encoded_Pclass=SparseVector(2, {0: 1.0}), indexed_Sex=0.0, encoded_Sex=SparseVector(1, {0: 1.0}), indexed_Ticket=202.0, encoded_Ticket=SparseVector(540, {202: 1.0}), indexed_Embarked=0.0, encoded_Embarked=SparseVector(2, {0: 1.0}), unscaled_features=SparseVector(549, {0: 1.0, 2: 1.0, 205: 1.0, 543: 1.0, 545: 22.0, 546: 1.0, 548: 7.25}), features=SparseVector(549, {0: 0.043, 2: 0.043, 205: 0.043, 543: 0.043, 545: 0.9454, 546: 0.043, 548: 0.3115}), label=0.0)

In [0]:
from pyspark.sql.functions import col
columns_to_train = labeled_train_df1.select('label', 'features')

In [29]:
columns_to_train.head()

Row(label=0.0, features=SparseVector(549, {0: 0.043, 2: 0.043, 205: 0.043, 543: 0.043, 545: 0.9454, 546: 0.043, 548: 0.3115}))

In [30]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from keras.optimizers import Adam

from elephas.ml_model import ElephasEstimator

Using TensorFlow backend.


WARNING


In [31]:
num_classes = columns_to_train.select('label').distinct().count()
print(num_classes)

2


In [32]:
input_dim = len(columns_to_train.select('features').first()[0])
print(input_dim)

549


In [33]:
model = Sequential()
model.add(Dense(256, input_shape=(input_dim,), activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(256, activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(num_classes))
model.add(Activation('relu'))
model.compile(loss='binary_crossentropy', optimizer='adam')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               140800    
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                

In [35]:
optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

estimator = ElephasEstimator()
estimator.setFeaturesCol('features')
estimator.setLabelCol('label')
estimator.set_keras_model_config(model.to_yaml())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(num_classes)
estimator.set_num_workers(1)
estimator.set_epochs(25) 
estimator.set_batch_size(64)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_877b997a5eb2

In [36]:
estimation = estimator.fit(columns_to_train)

>>> Fit model
>>> Synchronous training complete.


In [0]:
pred_train = estimation.transform(columns_to_train)


In [38]:
pred_train.filter(pred_train.label == pred_train.prediction).count()

674

In [39]:
pred_train.count()

712

In [40]:
print('The training accuracy is:', 656/714)

The training accuracy is: 0.9187675070028011


In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics


Test data needs dummy labels to create the same data structure as the train data.

In [0]:
from pyspark.sql.functions import lit
transformed_train_df1_label = transformed_train_df1.withColumn('Survived', lit(0))

In [48]:
transformed_train_df1_label.head()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S', indexed_Pclass=0.0, encoded_Pclass=SparseVector(2, {0: 1.0}), indexed_Sex=0.0, encoded_Sex=SparseVector(1, {0: 1.0}), indexed_Ticket=202.0, encoded_Ticket=SparseVector(540, {202: 1.0}), indexed_Embarked=0.0, encoded_Embarked=SparseVector(2, {0: 1.0}), unscaled_features=SparseVector(549, {0: 1.0, 2: 1.0, 205: 1.0, 543: 1.0, 545: 22.0, 546: 1.0, 548: 7.25}), features=SparseVector(549, {0: 0.043, 2: 0.043, 205: 0.043, 543: 0.043, 545: 0.9454, 546: 0.043, 548: 0.3115}))

In [0]:
index_label = StringIndexer(inputCol = 'Survived', outputCol = 'label')
labeled_test_df1 = index_label.fit(transformed_train_df1_label).transform(transformed_train_df1_label)

In [50]:
labeled_test_df1.head()

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Ticket='A/5 21171', Fare=7.25, Cabin=None, Embarked='S', indexed_Pclass=0.0, encoded_Pclass=SparseVector(2, {0: 1.0}), indexed_Sex=0.0, encoded_Sex=SparseVector(1, {0: 1.0}), indexed_Ticket=202.0, encoded_Ticket=SparseVector(540, {202: 1.0}), indexed_Embarked=0.0, encoded_Embarked=SparseVector(2, {0: 1.0}), unscaled_features=SparseVector(549, {0: 1.0, 2: 1.0, 205: 1.0, 543: 1.0, 545: 22.0, 546: 1.0, 548: 7.25}), features=SparseVector(549, {0: 0.043, 2: 0.043, 205: 0.043, 543: 0.043, 545: 0.9454, 546: 0.043, 548: 0.3115}), label=0.0)

In [0]:
columns_to_test = labeled_test_df1.select('label', 'features')

In [0]:
pred_test = estimation.transform(columns_to_test)

In [59]:
pred_test.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       0.0|  396|
|       1.0|  316|
+----------+-----+



Even though I assigned all 0 to the 'Survived' column in the test data, the deep learning model is able to predict 396 as 0 and 316 as 1 so it has prediction power. 